# **Setting Up RMI Depreciation Flags for FERC 1 Data**

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import sys
import os
import pathlib

# 3rd party libraries
import geopandas as gpd
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

In [3]:
# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [225]:
# Display settings
sns.set()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 20

In [5]:
# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq='AS'
)

In [6]:
# testing
def mini_df(df, pid, year):
    df1 =  df[df['plant_id_pudl']==pid] 
    
    if 'report_year' in df.columns:
        return df1[df1['report_year']==year]
    elif 'report_date' in df.columns:
        return df1[df1['report_date'].dt.year==year]

In [7]:
ferc_agg_cols = [
    'capacity_mw',
    'net_generation_mwh',
    'avg_num_employees',
    'capex_land', 'capex_equipment', 'capex_structures', 'capex_total', 'asset_retirement_cost',
    'opex_operations', 'opex_fuel', 'opex_coolants', 'opex_steam', 'opex_steam_other', 'opex_transfer', 'opex_electric', 'opex_misc_power', 'opex_rents', 'opex_allowances', 'opex_engineering', 'opex_structures', 'opex_boiler', 'opex_plants', 'opex_misc_steam', 'opex_production_total'
]

## **Step 1:** Create a better technology type

### **Step 1.1:** Merge steam table with fbp table
PUDL already merges the steam and fuel tables in the fbp (fuel_by_plant) table. This table classifies each plant's fuel type based on fuel cost and fuel consumed by each plant by each fuel type. The fuel type with the highest percentage (according to a given threshold) is deemed the "primary" fuel source.

Here, we merge the fbp table with the steam to get a new table that includes the primary fuel types and the steam fields excluded from the fbp table.

In [1246]:
# Generate the tables we'll be working with
fbp = pudl_out.fbp_ferc1()
steam = pudl_out.plants_steam_ferc1()
plants = pudl_out.plants_eia860()
gens = pudl_out.gens_eia860()
net_gen = pudl_out.gen_eia923()
fuel = pudl_out.fuel_ferc1()

# Specify some column groupings for later
ferc_id_cols = ['report_year', 'utility_id_ferc1', 'plant_name_ferc1']
gen_id_cols = ['report_date', 'plant_id_pudl', 'generator_id']

In [1268]:
# Merge fbp and steam tables
fbp_sample = fbp[ferc_id_cols + ['primary_fuel_by_mmbtu', 'primary_fuel_by_cost']]
steam_better_fuel = pd.merge(steam, fbp_sample, on=ferc_id_cols, how='left').drop_duplicates()

# Add primary fuel column and flag column to say where the fuel type designation comes from
steam_better_fuel['primary_fuel'] = (
    steam_better_fuel['primary_fuel_by_mmbtu']
    .replace({'':np.nan, 'unknown': np.nan})
)
steam_better_fuel.loc[steam_better_fuel.primary_fuel.notna(), 'primary_fuel_flag'] = 'fbp primary_fuel_by_mmbtu'

In [1269]:
print('rows with no fuel / total rows')
print(len(steam_better_fuel[steam_better_fuel['primary_fuel'].isna()]), '/', len(steam_better_fuel))

rows with no fuel / total rows
5096 / 29270


In [12]:
# *****This might not be what we want if it is plant-level!*****

# Extend primary plant fuel type designation to all rows in a given plant-year group
# steam_better_fuel['primary_plant_fuel_type'] = steam_better_fuel['primary_fuel_by_mmbtu'].astype('string')
# steam_better_fuel['primary_plant_fuel_type'] = (
#     steam_better_fuel
#     .groupby(['report_year', 'plant_id_pudl'])['primary_plant_fuel_type']
#     .transform(lambda x: x.max())
#     .replace([''], pd.NA)
# )

### **Step 1.2:** Merge steam with EIA

In [1250]:
# Combine generation from 860 with generation from 923 to get fuel type and net generation
net_gen_fuel = pd.merge(gens, net_gen[gen_id_cols+['net_generation_mwh']], on=gen_id_cols, how='outer')

In [1251]:
# Aggregate gen by plant and get primary plant fuel type based on net generation

# Get plant net generation sums
net_gen_fuel['plant_net_gen'] = (
    net_gen_fuel.groupby(['report_date', 'plant_id_pudl'])['net_generation_mwh'].transform('sum')
)

# Calculate the portion of net generation that each row is responsible for
net_gen_fuel['gen_pct'] = (
    net_gen_fuel['net_generation_mwh'] / net_gen_fuel['plant_net_gen'] * 100
)

# Calculate the sum portion of net generation attributable to each fuel type 
net_gen_fuel_sum = (
    net_gen_fuel.groupby(['report_date', 'plant_id_pudl', 'fuel_type_code_pudl'])['gen_pct'].sum().reset_index()
)

# Distinguish between rows that have more than one fuel (duplicates)
net_gen_fuel_sum['dup'] = net_gen_fuel_sum.duplicated(['report_date', 'plant_id_pudl'], keep=False)

# Keep rows with only one fuel
one_fuel = (
    net_gen_fuel_sum[net_gen_fuel_sum['dup']==False].copy()
    .rename(columns={'fuel_type_code_pudl': 'primary_fuel_type_eia'})
    .assign(report_year=lambda x: x.report_date.dt.year)
    .drop(['report_date', 'gen_pct', 'dup'], axis=1)
)

In [1258]:
# Combine steam table with EIA plant primary fuel type table, add flag to note fuel type location
steam_better_fuel_eia = pd.merge(steam_better_fuel, one_fuel, on=['report_year', 'plant_id_pudl'], how='left')

# Add EIA primary fuels to the primary_fuel column where null and flag location
steam_better_fuel_eia.loc[(steam_better_fuel_eia.primary_fuel.isna()) & (steam_better_fuel_eia.primary_fuel_type_eia.notna()), 'primary_fuel_flag'] = 'eia923 net_generation_mwh'
steam_better_fuel_eia['primary_fuel'] = (
    steam_better_fuel_eia['primary_fuel']
    .fillna(steam_better_fuel_eia['primary_fuel_type_eia'])
)

# Add by primary_fuel_by_cost to further null cols and flag col
steam_better_fuel_eia.loc[(steam_better_fuel_eia.primary_fuel.isna()) & (steam_better_fuel_eia.primary_fuel_by_cost.notna()), 'primary_fuel_flag'] = 'fbp primary_fuel_by_cost'
steam_better_fuel_eia['primary_fuel'] = (
    steam_better_fuel_eia['primary_fuel']
    .fillna(steam_better_fuel_eia['primary_fuel_by_cost'])
    .replace({'':np.nan, 'unknown': np.nan, 'other': np.nan})
)

In [1265]:
print('rows with no fuel / total rows')
print(len(steam_better_fuel_eia[steam_better_fuel_eia['primary_fuel'].isna()]), '/', len(steam_better_fuel))

rows with no fuel / total rows
2914 / 29270


### **Step 1.3:** Merge with unlimited FERC

In [1261]:
# Create a new FERC fuel table that doesn't extrac for fuel != '' and fuel_quantity > 0

ferc1_years = pudl.constants.working_partitions['ferc1']['years']
ferc1_tables = ['fuel_ferc1'] # plants_steam_ferc1 
ferc1_raw_dfs = pudl.extract.ferc1.extract(
        ferc1_tables=ferc1_tables,
        ferc1_years=ferc1_years,
        pudl_settings=pudl_settings)
ferc1_transformed_dfs = pudl.transform.ferc1.transform(
        ferc1_raw_dfs, ferc1_tables=ferc1_tables)
fuel_untamed = ferc1_transformed_dfs['fuel_ferc1']
fuel_untamed_raw = ferc1_raw_dfs['fuel_ferc1']

Converting extracted FERC Form 1 table fuel_ferc1 into a pandas DataFrame.
Transforming raw FERC Form 1 dataframe for loading into fuel_ferc1


In [1263]:
# Only grab the rows with fuel types that are known and not duplicated (have a single fuel)
fuel_untamed_with_fuel = fuel_untamed[fuel_untamed['fuel_type_code_pudl']!='unknown'].copy()
fuel_untamed_with_fuel['dup'] = fuel_untamed_with_fuel.duplicated(['report_year', 'utility_id_ferc1', 'plant_name_ferc1'], keep=False)
single_fuel = fuel_untamed_with_fuel[fuel_untamed_with_fuel['dup']==False][['report_year', 'utility_id_ferc1', 'plant_name_ferc1', 'fuel_type_code_pudl']]

# Combine with running table
steam_better_fuel_untamed = (
    pd.merge(
        steam_better_fuel_eia, single_fuel, 
        on=['report_year', 'utility_id_ferc1', 'plant_name_ferc1'],
        how='left')
)

# Add flag
steam_better_fuel_untamed.loc[
    (steam_better_fuel_untamed.primary_fuel.isna()) 
    & (steam_better_fuel_untamed.fuel_type_code_pudl.notna()), 
    'primary_fuel_flag'] = 'fuel_ferc1 no limits'

# Add to primary fuel
steam_better_fuel_untamed['primary_fuel'] = (
    steam_better_fuel_untamed['primary_fuel']
    .fillna(steam_better_fuel_untamed['fuel_type_code_pudl'])
)

In [1266]:
print('rows with no fuel / total rows')
print(len(steam_better_fuel_untamed[steam_better_fuel_untamed['primary_fuel'].isna()]), '/', len(steam_better_fuel))

rows with no fuel / total rows
2582 / 29270


### **Step 1.4:** Merge with raw, unconverted FERC fuel average heat
Now we're going to fill in the gaps. To do this, we need the raw FERC1 data without any unit conversions. This is because in the transformed data, only units with a stated fuel and unit type get unit conversions. This means that you can't compare the these units across time (if one year units and fuel are reported and another year they are not). It's likely that plans will report in the same units (regardless of whether they report a unit or fuel type), so we need the unconverted version to compare them and make inferences about the missing fuel types.

In [1270]:
# Perform a couple of simple cleaning mechanisms on the raw data to make it more compatible
# with the cleaned steam table.
fuel_ferc1_df = (
    pudl.transform.ferc1._clean_cols(ferc1_raw_dfs['fuel_ferc1'], 'f1_fuel').
    pipe(pudl.helpers.simplify_strings, ['plant_name']).
    pipe(pudl.helpers.cleanstrings, ['fuel', 'fuel_unit'],
         [FUEL_STRINGS, FUEL_UNIT_STRINGS],
         unmapped='')
)

In [1271]:
# Select rows with no null heat rates and create a row that indicates the amount (clarify what that means)
fah = (
    fuel_ferc1_df
    .loc[fuel_ferc1_df['fuel_avg_heat'].notna()]
    .copy()
    .rename(columns={'plant_name': 'plant_name_ferc1'})
    .assign(amount=lambda x: x.fuel_quantity * x.fuel_avg_heat)
)

# FILL IN
idx = fah.groupby(['report_year', 'utility_id_ferc1', 'plant_name_ferc1'])['amount'].transform('max') == fah['amount']
fah = fah[idx]
fah = fahh[['report_year', 'utility_id_ferc1', 'plant_name_ferc1', 'fuel', 'fuel_unit', 'fuel_avg_heat', 'fuel_quantity']]

# Combine with the main df
steam_better_fuel_infer = pd.merge(
    steam_better_fuel_untamed, fahh, 
    on=['report_year', 'utility_id_ferc1', 'plant_name_ferc1'], 
    how='left')

# Create a new column for the imputed fuel type
steam_better_fuel_infer['new_fuel'] = np.nan


In [964]:
#test_df = fahh[fahh['plant_name_ferc1'].isin(['beluga', 'gorgas'])].copy()
# test_df['new_fuel'] = np.nan
# #test_df.groupby(['plant_name_ferc1']).apply(lambda x: test(x))
# test_df = test_df[test_df['report_year'].isin(range(2014,2019))]
# test_df

In [1272]:
def create_dict(df):
    no_unk = df[(df['primary_fuel'].notna()) & (df['fuel_avg_heat'].notna())] # != 'unknown'
    no_unk = no_unk.groupby(['plant_name_ferc1', 'primary_fuel'])['fuel_avg_heat'].median().reset_index()
    no_unk['10pct'] = no_unk['fuel_avg_heat']*0.1

    fuel_dict = {}
    for i in no_unk['plant_name_ferc1'].unique():
        fuel_dict[i] = dict(zip(
            no_unk.loc[no_unk['plant_name_ferc1'] == i]['primary_fuel'], 
            no_unk.loc[no_unk['plant_name_ferc1'] == i]['fuel_avg_heat']
        ))
    # Turn the median values into ranges based on 10% buffer
    for k,v in fuel_dict.items():
        for kk,vv in v.items():
            v[kk] = range(int(vv-vv*0.1), int(vv+vv*0.1))
    #print(fuel_dict)
    return fuel_dict

def update_df(df):    
    f_dict = create_dict(df)
    for k,v in f_dict.items():
        dd = df.loc[df['plant_name_ferc1']==k]
        df['new_fuel'].update(dd['fuel_avg_heat'].apply(lambda x: next((kk for kk,vv in v.items() if x in vv), np.nan)))
    return df

In [1273]:
steam_better_fuel_infer = update_df(steam_better_fuel_infer)

# Add flag
steam_better_fuel_infer.loc[
    (steam_better_fuel_infer.primary_fuel.isna()) 
    & (steam_better_fuel_infer.new_fuel.notna()), 
    'primary_fuel_flag'] = 'fuel_ferc1 infer from heat content of years'

steam_better_fuel_infer['primary_fuel'].update(steam_better_fuel_infer['new_fuel'])

In [1274]:
print('rows with no fuel / total rows')
print(len(steam_better_fuel_infer[steam_better_fuel_infer['primary_fuel'].isna()]), '/', len(steam_better_fuel))

rows with no fuel / total rows
2475 / 29270


In [951]:
#test = steam_better_fuel_infer[steam_better_fuel_infer['plant_name_ferc1'].isin(['beluga', 'gorgas', 'gadsden'])].copy()

In [1277]:
steam2 = (
    steam_better_fuel_infer[[
        'report_year', 'utility_id_ferc1', 'utility_id_pudl',
        'plant_id_pudl', 'plant_id_ferc1', 'plant_name_ferc1'] 
        + ferc_agg_cols + 
       ['primary_fuel_by_mmbtu', 'primary_fuel_by_cost',
        'primary_fuel', 'primary_fuel_flag', 'record_id']].copy()
)

In [1278]:
nona = steam_better_fuel_infer[steam_better_fuel_infer['primary_fuel'].notna()]
na = steam_better_fuel_infer[steam_better_fuel_infer['primary_fuel'].isna()]

print('rows with no fuel / total rows')
print(len(steam_better_fuel_infer[steam_better_fuel_infer['primary_fuel'].isna()]), '/', len(steam_better_fuel_infer))

print('percent of total records')
print(3222/29270*100)
cap_full = steam_better_fuel_infer['capacity_mw'].sum()
cap_na = steam_better_fuel_infer[steam_better_fuel_infer['primary_fuel'].isna()]['capacity_mw'].sum()

print('percent of total capacity')
print(cap_na/cap_full*100)

print('total plants')
print(len(na['plant_id_pudl'].unique()))

print('total utilities')
print(len(na['utility_id_pudl'].unique()))

rows with no fuel / total rows
2475 / 29299
percent of total records
11.007857874957294
percent of total capacity
3.757649444177934
total plants
536
total utilities
153


In [137]:
# Cases where EIA and FERC fuel type don't match up perfectly 
test = steam_better_fuel[['report_year', 'plant_id_pudl', 'primary_fuel_by_mmbtu']]
tt = pd.merge(test, net_gen_fuel_50, on=['report_year', 'plant_id_pudl'], how='outer')
dd = tt[(tt['primary_fuel_type_eia'].notna()) & (tt['primary_fuel_by_mmbtu'].notna())].copy()
dd['diff'] = dd['primary_fuel_type_eia'] != dd['primary_fuel_by_mmbtu'].copy()
dl = dd[dd['diff']==True]
dl

,report_year,plant_id_pudl,primary_fuel_by_mmbtu,primary_fuel_type_eia,diff
245,2013,206,gas,coal,True
249,2017,206,,gas,True
250,2018,206,,gas,True
251,2019,206,,gas,True
277,2009,32,,coal,True
...,...,...,...,...,...
28958,2015,517,gas,waste,True
28959,2016,517,gas,waste,True
28960,2017,517,gas,waste,True
28961,2018,517,gas,waste,True


#### Things to do still: 
- historic back-filling
- [x] check fuel table transform to see if any values are dropped / changed
- add old eia923 years
- [x] look at Jon's detailed list of changes to fill these

## **Step 2:** Create flags for rows that represent TOTALS
Create flag columns for the steam table that notes which rows should be included when aggregating different data fields for each utility. These fields are: capacity, generation, number of employees, original cost, operation expenses.


The plan is to develop a flag to note whether a line is the whole plant or the owned portion

**Flags:**
- utility owned total
- unit total
- plant total
- combustion turbine extra

### Functions

In [975]:
def flag_totals(steam_table):
    """Preliminarily mark the rows with total indicated in the plant name."""
    regex1 = r'(?i)tot[a-z]*' # Find anything that would resembles TOTAL in the plant name
    regex2 = '100%' # Find all names with 100% in them
    regex3 = 'ttl ' # the space at the end is important
    regex4 = r'\(all' # find any names with '(all' in them
    
    steam_table['total'] = steam_table['plant_name_ferc1'].str.contains('|'.join([regex1, regex2, regex3, regex4]))
    ser_no_octo = steam_table[steam_table['plant_name_ferc1'].str.contains('octotillo')].total == False
    steam_table['total'].update(ser_no_octo) # remove octotillio from total = True because it accidentally fits in the 'tot' regex
    
    return steam_table

In [976]:
def flag_plant_totals(df, col_name):
    
    def is_plant_total(row):
        if 'total plant' in row:
            return 'plant total'
        elif 'plant total' in row:
            return 'plant total'
        elif 'total plt' in row:
            return 'plant total'
        elif 'ttl plt' in row:
            return 'plant total'
        elif 'tot. plt.' in row:
            return 'plant total'
        elif '100%' in row:
            return 'plant total'
        else:
            return None
    
    df[col_name] = df.apply(lambda x: is_plant_total(x.plant_name_ferc1), axis=1)
    
    return df

In [977]:
def backfill_years_by_capacity(df, col_name, replace, replace_with):
    """Backfill rows based on capacity."""
    
    only_totals_df = df[df[col_name]==replace_with]
    plant_groups = only_totals_df.groupby('plant_id_pudl')
    capacity_dict = plant_groups['capacity_mw'].apply(lambda x: [x for x in list(x.unique()) if x !=0]).to_dict() # no zeros
    plants_with_totals_list = list(capacity_dict.keys())
    
    for plant_id in plants_with_totals_list:
        for capacity in capacity_dict[plant_id]:
            one_plant_df = df[df['plant_id_pudl']==plant_id].copy()
            cap_match_df = one_plant_df[one_plant_df['capacity_mw'].isin(capacity_dict[plant_id])]
            series_update = cap_match_df[col_name].replace({replace: replace_with})
            df[col_name].update(series_update) 
            
    return df

In [978]:
def categorize_bad_rows(df, f_list): # could probably make this faster...
    """Flag bad rows."""
    for fix_dict in f_list:
        for year in fix_dict['years']:
            df.loc[df['record_id']==f"f1_steam_{year}{fix_dict['id_suffix']}", 'total_type'] = fix_dict['total_type']
    
    return df

In [979]:
# Fix-a-dic
# need to append ferc1_steam_YEAR_id_suffix
# categories:
# - plant total
# - unit total
# - utility owned total
# - combustion turbine extra

fix_list = [
    # Rockport AEP
    {'id_suffix': '_12_1_0_3', 'total_type': 'utility owned total', 'years': range(1994,2020)}, #pudl id 530
    # Rockport IMP
    {'id_suffix': '_12_73_1_3', 'total_type': 'utility owned total', 'years': range(1994,1997)}, #pudl id 530
    {'id_suffix': '_12_73_0_3', 'total_type': 'utility owned total', 'years': range(1997,2020)}, # pudl id 530
    # Amos APC
    {'id_suffix': '_12_6_0_3', 'total_type': 'plant total', 'years': range(1994,2002)}, #pudl id 16
    # Conesville 4 - Columbus Southern Power Company then Ohio Power Company then AEP
    {'id_suffix': '_12_31_0_3', 'total_type': 'unit total', 'years': range(1994,2011)}, # pudl id 128
    {'id_suffix': '_12_127_4_1', 'total_type': 'unit total', 'years': range(2011,2014)}, # pudl id 128
    {'id_suffix': '_12_452_1_2', 'total_type': 'unit total', 'years': range(2014,2015)}, # pudl id 128
    # Conesville 4 - Duke 
    {'id_suffix': '_12_27_1_3', 'total_type': 'unit total', 'years': range(1994,2003)}, # pudl if 128  # was plant total
    # Belle River - DTE
    {'id_suffix': '_12_44_0_1', 'total_type': 'utility owned total', 'years': range(1994, 2020)}, # pudl id 44  # also plant total, doesn't add up first year
    # Mitchell - Kentucky Power
    {'id_suffix': '_12_81_0_3', 'total_type': 'plant total', 'years': range(2014,2020)}, # pudl id 382  
    # Mitchell - AEP then Wheeling Power
    {'id_suffix': '_12_452_3_3', 'total_type': 'plant total', 'years': range(2014,2015)}, # pudl id 382
    {'id_suffix': '_12_192_0_2', 'total_type': 'plant total', 'years': range(2015,2020)}, # pudl id 382
    # Iatan 1 - Kansas City Power and Light
    {'id_suffix': '_12_79_1_1', 'total_type': 'unit total', 'years': range(2010,2020)}, # pudl id 295  # was plant total
    # Iatan 2 - Kansas Ciry Power and Light
    {'id_suffix': '_12_79_1_3', 'total_type': 'unit total', 'years': range(2010,2020)}, # pudl id 295  # was plant total
    # La Cygne - Kansas  
    #{'id_suffix': '_12_80_0_3', 'total_type': 'plant total', 'years': range(1994,2010)}, # pudl id 336  # very weird, nums don't add up
    # Jeffrey - Kansas Gas and Electric
    {'id_suffix': '_12_80_1_1', 'total_type': 'plant total', 'years': range(1994,1995)}, # pudl id 307
    {'id_suffix': '_12_80_1_3', 'total_type': 'plant total', 'years': range(1995,2002)}, # pudl id 307
    {'id_suffix': '_12_80_1_1', 'total_type': 'plant total', 'years': range(2002,2010)},
    # Jeffrey - Westar Energy
    {'id_suffix': '_12_191_1_4', 'total_type': 'plant total', 'years': range(1994,1995)}, # pudl id 307
    {'id_suffix': '_12_191_1_5', 'total_type': 'plant total', 'years': range(1995,2005)}, # pudl id 307
    {'id_suffix': '_12_191_1_3', 'total_type': 'plant total', 'years': range(2005,2006)}, # pudl id 307
    {'id_suffix': '_12_191_1_5', 'total_type': 'plant total', 'years': range(2006,2010)}, # pudl id 307
    # JM Stuart - Duke Energy
    {'id_suffix': '_12_27_1_1', 'total_type': 'plant total', 'years': range(1994,2003)}, # pudl id 288
    # JM Stuart - Dayton Power and Light
    {'id_suffix': '_12_42_2_1', 'total_type': 'plant total', 'years': range(1994,1998)}, # pudl id 288
    {'id_suffix': '_12_42_1_1', 'total_type': 'plant total', 'years': range(1998,2001)}, # pudl id 288
    # Valley - Wisconsin Power and Electric
    {'id_suffix': '_12_193_0_3', 'total_type': 'utility owned total', 'years': range(1994,1996)}, # pudl id 603  # could also be plant total
    # Pt. Wash - Wisconsin Electric Power
    {'id_suffix': '_12_193_1_4', 'total_type': 'utility owned total', 'years': range(1994,1996)}, # pudl id 470  # other weird value -- see below
    # Pt. Wash (gas) - Wisconsin Electric Power
    {'id_suffix': '_12_193_7_1', 'total_type': 'combustion turbine extra', 'years': range(1994,1996)}, # pudl id 470
    {'id_suffix': '_12_193_1_3', 'total_type': 'combustion turbine extra', 'years': range(1996,2004)}, # pudl id 470  # 2004 might not be right -- confusing
    
    {'id_suffix': '_12_193_3_4', 'total_type': 'utility owned total', 'years': range(1994,1996)}, # pudl id 469  # also plant total
    
    {'id_suffix': '_12_193_4_4', 'total_type': 'utility owned total', 'years': range(1994,1996)}, # pudl id 542  # also plant total
    {'id_suffix': '_12_193_0_4', 'total_type': 'utility owned total', 'years': range(1996,2008)}, # has 1 unit only here on...
    {'id_suffix': '_12_193_1_3', 'total_type': 'utility owned total', 'years': range(2008,2015)}, # pudl id 542  # technically becomes just one row in 2010
    
    {'id_suffix': '_12_193_5_2', 'total_type': 'utility owned total', 'years': range(1994,1996)}, # pudl id 1216 # also plant total
    {'id_suffix': '_12_193_0_5', 'total_type': 'utility owned total', 'years': range(1996,2008)}, # pudl id 1216 # has 1 unit only
    
    {'id_suffix': '_12_193_5_5', 'total_type': 'utility owned total', 'years': range(1994,1996)}, # pudl id 458 # also plant total
    
    {'id_suffix': '_12_193_6_5', 'total_type': 'utility owned total', 'years': range(1994,1996)}, # pudl id 216  # also plant total
    
    {'id_suffix': '12_193_8_4', 'total_type': 'utility owned total', 'years': range(1994,1996)}, # pudl id 127  # also plant total
    
    {'id_suffix': '_12_194_0_4', 'total_type': 'unit total', 'years': range(1994,2012)}, # pudl id 123  # was labeled plant total b/c of 100%
    {'id_suffix': '_12_194_0_1', 'total_type': 'unit total', 'years': range(2012,)}, # pudl id 123
    
    {'id_suffix': '_12_194_0_5', 'total_type': 'unit total', 'years': range(1994,2012)}, # pudl id 123  # was labeled plant total b/c of 100%
    {'id_suffix': '_12_194_0_2', 'total_type': 'unit total', 'years': range(2012,)}, # pudl id 123
    
    {'id_suffix': '_12_194_2_3', 'total_type': 'utility owned total', 'years': range(1994,1998)}, # pudl id 123 
    {'id_suffix': '_12_194_1_4', 'total_type': 'utility owned total', 'years': range(1998,2012)}, # pudl id 123  # continuation of before 2_3 to 1_4
    {'id_suffix': '_12_194_1_1', 'total_type': 'utility owned total', 'years': range(2012,2014)}, # pudl id 123 
    
    {'id_suffix': '_12_89_1_5', 'total_type': 'utility owned total', 'years': range(1994,1998)},# pudl id 123
    {'id_suffix': '_12_89_1_3', 'total_type': 'utility owned total', 'years': range(1998,2002)}, # pudl id 123  # continuation of before 1_5 to 1_3
    {'id_suffix': '_12_89_1_5', 'total_type': 'utility owned total', 'years': range(2002,2007)}, # pudl id 123  # record id went back from 1_3 to 1_5
    {'id_suffix': '_12_89_0_4', 'total_type': 'utility owned total', 'years': range(2007,2008)}, # pudl id 123  # 1_5 to 0_4
    {'id_suffix': '_12_89_0_5', 'total_type': 'utility owned total', 'years': range(2008,2009)}, # pudl id 123  # 0_4 to 0_5
    {'id_suffix': '_12_89_1_4', 'total_type': 'utility owned total', 'years': range(2010,2012)}, # pudl id 123  # 0_5 to 1_4
    {'id_suffix': '_12_89_0_5', 'total_type': 'utility owned total', 'years': range(2012,2014)}, # pudl id 123
    {'id_suffix': '_12_89_1_4', 'total_type': 'utility owned total', 'years': range(2014,2015)}, # pudl id 123
    {'id_suffix': '_12_89_1_5', 'total_type': 'utility owned total', 'years': range(2015,2016)}, # pudl id 123
    {'id_suffix': '_12_89_0_5', 'total_type': 'utility owned total', 'years': range(2016,2019)}, # pudl id 123
    {'id_suffix': '_12_89_0_4', 'total_type': 'utility owned total', 'years': range(2019,2020)}, # pudl id 123
    
    {'id_suffix': '_12_194_2_5', 'total_type': 'unit total', 'years': range(1994,1998)}, # pudl id 171  # was plant total
    {'id_suffix': '_12_194_2_1', 'total_type': 'unit total', 'years': range(1998,2012)}, # pudl id 171
    {'id_suffix': '_12_194_1_3', 'total_type': 'unit total', 'years': range(2012,2014)}, # pudl id 171
    {'id_suffix': '_12_194_1_1', 'total_type': 'unit total', 'years': range(2014,2016)}, # pudl id 171
    {'id_suffix': '_12_194_0_5', 'total_type': 'unit total', 'years': range(2016,2019)}, # pudl id 171
    
    {'id_suffix': '_12_194_3_4', 'total_type': 'unit total', 'years': range(1994,1998)}, # pudl id 171  # contains unit-1, was plant total
    {'id_suffix': '_12_194_2_3', 'total_type': 'unit total', 'years': range(1998,2012)}, # pudl id 171
    {'id_suffix': '_12_194_1_5', 'total_type': None, 'years': range(2012,2014)}, # pudl id 171 -- was 2_3 but now 1_5 and owned by one utility   
    
    {'id_suffix': '_12_134_0_5', 'total_type': 'utility owned total', 'years': range(1994,2001)}, # pudl id 281  # also plant total
    {'id_suffix': '_12_134_1_5', 'total_type': 'utility owned total', 'years': range(2001,2016)},
    {'id_suffix': '_12_134_1_4', 'total_type': 'utility owned total', 'years': range(2016,2020)},
    
    {'id_suffix': '_12_138_0_5', 'total_type': 'utility owned total', 'years': range(1994,)}, # pudl id 2281  # also plant total
    
    {'id_suffix': '_12_138_2_3', 'total_type': 'combustion turbine extra', 'years': range(1994,1997)}, # pudl id 2281  # pause for 2 years
    {'id_suffix': '_12_138_4_1', 'total_type': 'combustion turbine extra', 'years': range(1999,2000)}, # pudl id 2281
    {'id_suffix': '_12_138_2_3', 'total_type': 'combustion turbine extra', 'years': range(2000,2001)}, # pudl id 2281

    {'id_suffix': '_12_195_1_5', 'total_type': 'utility owned total', 'years': range(1994,2008)}, # pudl id 503  # was plant total, is also technically plant total...
    {'id_suffix': '_12_195_1_3', 'total_type': 'utility owned total', 'years': range(2008,2011)}, # pudl id 503
    {'id_suffix': '_12_195_1_1', 'total_type': 'utility owned total', 'years': range(2011,2019)}, # pudl id 503
   
    {'id_suffix': '_12_195_3_4', 'total_type': 'utility owned total', 'years': range(1994,2011)}, # pudl id 473  # was plant total, might also be plant total
    {'id_suffix': '_12_195_2_3', 'total_type': 'utility owned total', 'years': range(2011,2018)}, # pudl id 473
    {'id_suffix': '_12_195_2_2', 'total_type': 'utility owned total', 'years': range(2018,2019)}, # pudl id 473 # but there is wierdness with the w31 and w32
    
    {'id_suffix': '_12_195_2_5', 'total_type': 'unit total', 'years': range(2008,2011)}, # pudl id 473
    {'id_suffix': '_12_195_2_1', 'total_type': 'unit total', 'years': range(2011,2018)}, # pudl id 473
    
    {'id_suffix': '_12_195_3_5', 'total_type': 'plant total', 'years': range(1994,2006)}, # pudl id 1166
    
    {'id_suffix': '_12_195_5_5', 'total_type': 'unit total', 'years': range(2004,2006)}, # pudl id 343
    {'id_suffix': '_12_195_5_3', 'total_type': 'unit total', 'years': range(2006,2011)}, # pudl id 343
    {'id_suffix': '_12_195_3_3', 'total_type': 'plant total', 'years': range(2018,2019)}, # pudl id 343

    {'id_suffix': '_12_57_5_1', 'total_type': 'utility owned total', 'years': range(1994,1995)}, # pudl id 661  # was plant total  # doesn't add up
    {'id_suffix': '_12_57_4_1', 'total_type': 'utility owned total', 'years': range(1995,1998)}, # pudl id 661  # skips a year
    {'id_suffix': '_12_57_4_1', 'total_type': 'utility owned total', 'years': range(1999,2009)}, # pudl id 661

    {'id_suffix': '_12_57_5_3', 'total_type': 'utility owned total', 'years': range(1994,1995)}, # pudl id 257  # was plant total
    {'id_suffix': '_12_57_5_1', 'total_type': 'utility owned total', 'years': range(1995,2009)}, # pudl id 257  # doesn't add up

    {'id_suffix': '_12_193_9_4', 'total_type': 'utility owned total', 'years': range(1995,1996)}, # pudl id 443

    {'id_suffix': '_12_281_0_2', 'total_type': 'utility owned total', 'years': range(2002,2007)}, # pudl id 1110  # was plant total and maybe is

    {'id_suffix': '_12_89_2_5', 'total_type': 'utility owned total', 'years': range(2011,2019)},
    {'id_suffix': '_12_89_1_2', 'total_type': 'utility owned total', 'years': range(2019,2020)},
]

# pudl id 336 unclear which is the total
# pudl id 652 is fishy and kind of seems like a duplicate?
# pudl id 40 needs some attention....
# pudl id 410 unsure role of cge
# pudl id 167 unsure role of cge
# pudl id 316 unsure role of cge
# pudl id 611 unsure role of cge
# pudl id 470 in 2008 two totals?
# pudl id 363 gets confusing around 2008 
# pudl id 281 unt 2 in year 1999 might get double counted
# pudl id 1209 components don't add up
# pudl id 503 pulliam-common? with capcity 0 and in ~2004 pulliam 31 shows up
# pudl id 473 has "communal" row as well and in ~1997 w31, w32
# pudl id 661 is confusing which values are which
# pudl id 529 doesn't add up
# pudl id 610 confused by what this 100% ownership thing is...
# pudl id 90 confusing
# pudl id 183 confusing total value in 2011

In [980]:
def compare_totals(flag_df, comp_col):
    """Sum non-total utility-plant reported values and compare to reported totals when applicable.
    
    This function takes in a DataFrame and calculates the sum value for both the plant and utility-plant
    groups on an annual basis and compares them with any reported totals. First, this function calculates
    group totals by excluding any rows flagged as totals or extraneous in the total_types row (i.e.:
    total_type.isna()). Then, it separates the DataFrame into two small DataFrames representing all
    the utility owned total rows and all of the plant total rows reported and flagged. Next, it
    compares the calculated group total against the reported group total and outputs a True/False
    boolean. The boolean flags from these small table comparisons are then subsumed into the larger 
    DataBase under the column names: utility_owned_total_flag and plant_total_flag.
    
    These columns will show users where there might be a reporting discrepancy and allows them to
    choose which value they might want to reply on for further calculation. This flag is not yet
    incorporated into the aggregation function below, but it could be!
    
    """
    def sum_no_totals(df, col):
        no_totals_df = df.loc[df['total_type'].isna()]
        return no_totals_df[col].sum()

    # Groupby utility-plant and plant
    plant_util_groups = flag_df.groupby(['report_year', 'utility_id_pudl', 'plant_id_pudl'])
    plant_groups = flag_df.groupby(['report_year', 'plant_id_pudl'])

    # Get the sum of the values in each group that are not totals
    plant_util_total_series = plant_util_groups.apply(lambda x: sum_no_totals(x, comp_col))
    plant_total_series = plant_groups.apply(lambda x: sum_no_totals(x, comp_col))

    # Take that series of sums, reset the index, and give the sum column an informative name
    util_plant_df = pd.DataFrame(plant_util_total_series).reset_index().rename(columns={0:'plant_util_total'})
    plant_df = pd.DataFrame(plant_total_series).reset_index().rename(columns={0:'plant_total'})

    # Merge the two sum columns together on utility-plant
    df = pd.merge(plant_df, util_plant_df, on=['report_year', 'plant_id_pudl'], how='outer')
    comp_totals = pd.merge(flag_df, df, on=['report_year', 'utility_id_pudl', 'plant_id_pudl'], how='outer')
    
    # Get slices of the dataframe that represent the utility owned total rows and the plant total rows to compare with calculated totals
    util_ot = comp_totals[comp_totals['total_type']=='utility owned total'].copy()
    plant_ot = comp_totals[comp_totals['total_type']=='plant total'].copy()

    # For each dataframe, flag whether the sum of the components (calculated above) equals the reported total
    # Could make this a little more flexible (+/- 1)
    util_ot['utility_owned_total_flag'] = np.where(util_ot['capacity_mw'] == util_ot['plant_util_total'], True, False)
    plant_ot['plant_total_flag'] = np.where(plant_ot['capacity_mw'] == plant_ot['plant_total'], True, False)
    
    # Create blank columns in the original dataframe
    comp_totals[f'{comp_col}_utility_owned_total_flag'] = None
    comp_totals[f'{comp_col}_plant_total_flag'] = None

    # Update those blank columns so that the True/False values from the prior calculations are added to the correct column
    comp_totals[f'{comp_col}_utility_owned_total_flag'].update(util_ot['utility_owned_total_flag'])
    comp_totals[f'{comp_col}_plant_total_flag'].update(plant_ot['plant_total_flag'])

    return comp_totals

### **Step 2.1:** Add a simple totals flag to the steam table

In [1320]:
# Create copy of the steam table with fuel types merged in
steam_test = steam2.copy()

# Flag bad/total rows
flagged_steam = (
     steam_test
     .pipe(flag_totals)
     .pipe(backfill_years_by_capacity, col_name='total', replace=False, replace_with=True)
     .pipe(flag_plant_totals, col_name='total_type')
     .pipe(backfill_years_by_capacity, col_name='total_type', replace=None, replace_with='plant total')
     .pipe(categorize_bad_rows, f_list=fix_list)
     .drop('total', axis=1)
     .assign(primary_fuel = lambda x: x.primary_fuel.fillna('unknown'))
)

In [1299]:
#flagged_steam['primary_fuel'] = flagged_steam['primary_fuel'].replace({'':'unknown', 'other': 'unknown'})

In [1321]:
flagged_steam['primary_fuel'].unique()

array(['unknown', 'coal', 'gas', 'nuclear', 'oil', 'waste', 'wind',
       'solar', 'hydro'], dtype=object)

### **Step 2.2 (optional):** Check reported totals against sum of available components
For utility-owned plant portions and entire plants

In [999]:
# Compare the reported totals with the sum of the reported components
flagged_steam_total_comp = compare_totals(flagged_steam, 'capacity_mw')

In [1002]:
# Find miss-matching utility owned totals
uot = flagged_steam_total_comp[flagged_steam_total_comp['total_type']=='utility owned total']
uot_bad = uot[uot['capacity_mw_utility_owned_total_flag']==False]
print(int((len(uot_bad)/len(uot)*100)),'% of reported utility owned total values differ from the sum of their reported components')
uot_bad['plant_id_pudl'].unique()

53 % of reported utility owned total values differ from the sum of their reported components


array([  44,  542, 1216,  281,  503,  473,  661,  257, 1110,  123])

In [1003]:
# Find miss-matching plant totals
pt = flagged_steam_total_comp[flagged_steam_total_comp['total_type']=='plant total']
pt_bad = pt[pt['capacity_mw_plant_total_flag']==False]
print(int(len(pt_bad)/len(pt)*100),'% of reported plant total values differ from the sum of their reported components')
pt_bad['plant_id_pudl'].unique()

64 % of reported plant total values differ from the sum of their reported components


array([  16,  450,  288, 1087,  295,  307,  652, 1665,  123, 1209, 1166,
        529,  610,  473,  343,  336])

### **Step 2.3:** Custom aggregation based on the presense of nulls and/or totals rows
Most of the time the total rows are excluded. Sometimes, however, they provide valuable information we might want to use in the aggregation of certain columns.

The fields we'd like to aggregate on are: 
- capacity_mw
- net_generation_mwh
- avg_num_employees
- **original cost:** capex_land, capex_equipment, capex_structures, capex_total, asset_retirement_cost
- **operational expenses:** opex_operations, opex_fuel, opex_coolants, opex_steam, opex_steam_other, opex_transfer, opex_electric, opex_misc_power, opex_rents, opex_allowances, opex_engineering, opex_structures, opex_boiler, opex_plants, opex_misc_steam, opex_production_total.

In [1322]:
agg_fields = [
    'capacity_mw', 
    'net_generation_mwh',
    'avg_num_employees',
    'capex_land',
    'capex_equipment',
    'capex_structures',
    'capex_total',
    'asset_retirement_cost',
    'opex_misc_power',
    'opex_rents',
    'opex_allowances',
    'opex_engineering',
    'opex_structures',
    'opex_boiler',
    'opex_plants',
    'opex_misc_steam',
    'opex_production_total'
]

In [1323]:
def col_aggregator(flag_df, agg_col):
    """
    Remove total rows from aggregation; use when there are nulls present in non-total rows. 
    
    This function looks at a table grouped by year, utility, and plant and determines whether to use any of the
    information from the total rows in an aggregation based on the column agg_col specified as a parameter.
    If there are no total rows, this function simply takes the sum of each group. If there are totals rows, 
    this function first looks to see if there is a utility owned total reported and then looks to see if there is
    a plant total reported.
    
    If the aggregation must resort to using a total row, a flag is created and returned alongside the aggregated value.
    These values will later get split apart and set as seperate columns (in the build_col_agg_df function).
    
    Args: 
        df (pandas.DataFrame): A flagged version of the cleaned ferc1_steam table with the column name 'total_type'
            specifying wither it is a utility owned total, unit total, or plant total.
        agg_col (str): The name of the column you'd like to aggregate by.
    
    """
    if flag_df.loc[flag_df['total_type'].isna()][agg_col].notna().all() & len(flag_df.loc[flag_df['total_type'].isna()]) > 0: 
            flag = None
            agg_value = flag_df.loc[flag_df['total_type'].isna()][agg_col].sum()
            return [agg_value, flag]
    else:
        if flag_df['total_type'].str.contains('utility owned total').any() & flag_df.loc[flag_df['total_type']=='utility owned total'][agg_col].notna().all():
            flag = 'used utility owned total'
            agg_value = flag_df.loc[flag_df['total_type']=='utility owned total'][agg_col].unique()[0]
            return [agg_value, flag]
        elif flag_df['total_type'].str.contains('plant total').any() & flag_df.loc[flag_df['total_type']=='plant total'][agg_col].notna().all():
            flag = 'used plant total pertains to more than one utility'
            agg_value = flag_df.loc[flag_df['total_type']=='plant total'][agg_col].unique()[0]
            return [agg_value, flag]
        elif flag_df.loc[flag_df['total_type'].isna()][agg_col].isna().all():
            flag = None
            agg_value = np.nan
            return [agg_value, flag]
        else:
            flag = 'aggregated with some null values'
            agg_value = flag_df.loc[flag_df['total_type'].isna()][agg_col].sum()
            return [agg_value, flag]

In [1324]:
def build_col_agg_df(flag_df, agg_col):
    """Sort by field level.
    
    This function creates a mini aggregated dataframe based on a column specified in the parameters. 
    It runs the col_aggregator function so that the aggregations exclude total values unless there are
    gaps in the subcomponents--in which case it will first try and use a reported utility total and 
    then a plant total. The col_aggregator function returns both the aggregated value and flag to
    indicate whether it was aggregated based on a value from one of the total rows. This function turns
    the value and the flag (returned as a list within one column) to seperate columns.
    
    These column-based data aggregations can later be merged to form one large data aggregation column.
    
    Args: 
        df (pandas.DataFrame): A DataFrame....
        agg_col (str): The name of the column you'd like to aggregate by.
    Returns:
        pandas.DataFrame: A DataFrame with rows selected based on priority.
    
    """

    plant_util_group = flag_df.groupby(['report_year', 'utility_id_pudl', 'plant_id_pudl', 'primary_fuel'])
    col_agg_series = plant_util_group.apply(lambda x: col_aggregator(x, agg_col))
    col_agg_df = pd.DataFrame(col_agg_series).reset_index()
    col_agg_df[[agg_col, f'{agg_col}_flag']] = pd.DataFrame(col_agg_df[0].tolist(), index=col_agg_df.index)
    col_agg_df = col_agg_df.drop(columns=[0])
    
    def combine_flags(ser):
        if ser.isna().all():
            return None
        else:
            return ', '.join([flag for flag in ser.unique() if flag != None])

    #Group by utility and fuel type
    util_fuel_df = (
        col_agg_df
        .groupby(['report_year', 'utility_id_pudl', 'primary_fuel'])
        .agg({agg_col: 'sum',
              f'{agg_col}_flag': lambda x: combine_flags(x)})
        .reset_index()
    )
    
    return util_fuel_df

In [1325]:
def aggregate_all_columns(df, col_list):
    agg_df = pd.DataFrame(columns=['report_year', 'utility_id_pudl', 'primary_fuel'])
    for col in col_list:
        col_df = build_col_agg_df(df, col)
        agg_df = pd.merge(agg_df, col_df, on=['report_year', 'utility_id_pudl', 'primary_fuel'], how='outer')
    return agg_df

In [1326]:
complete_util_agg = aggregate_all_columns(flagged_steam, agg_fields)

In [1336]:
merge_df.to_excel('/Users/aesharpe/Desktop/flag_comparison.xlsx')

In [1335]:
merge_df['primary_fuel'] = merge_df['primary_fuel'].replace({np.nan: 'unknown'})

In [1330]:
complete_util_agg.to_excel('/Users/aesharpe/Desktop/FERC1_utility_fuel.xlsx')

#### Things to do still: 
- if use a value from plant total, make it apply to the other utilities that also have a stake in that plant (avg_num_employees is a good example)
- if there is a value within 1 of a labeled value in the same plant group, it's probably a total
- add more values to the ones that get flagged
- check my flagged values against jon's
- combine aggregated column dfs into one big df? (but maybe not that necessary/hard
- change the way that flags work
- figure out what to do with unit totals (whether it's enough to just leave them marked) 
- figure out what to do with gas plants and other weird extra values
- aggregate by utility! That's what the end goal is afterall

In [1331]:
# read in Jon's CSV
jon_df = pd.read_csv('/Users/aesharpe/Desktop/Work/Catalyst_Coop/RMI/Depreciation/f1_steam_flagged_with_plant_id.csv')
steam3 = flagged_steam.copy()

jon_df = jon_df[['record_id', 'Flag', 'plant_name', 'report_year', 'plant_id_ferc1']].copy()
jon_df['plant_name'] = jon_df.plant_name.str.lower()

jon_df['rec'] = jon_df.report_year.map(str) + '-' + jon_df.plant_name.map(str)
steam3['rec'] = steam3.report_year.map(str) + '-' + steam3.plant_name_ferc1.map(str)

merge_df = pd.merge(steam_test, jon_df, on=['report_year','record_id'], how='left')

In [250]:
FUEL_STRINGS = {
    "coal": [
        'coal', 'coal-subbit', 'lignite', 'coal(sb)', 'coal (sb)', 'coal-lignite',
        'coke', 'coa', 'lignite/coal', 'coal - subbit', 'coal-subb', 'coal-sub',
        'coal-lig', 'coal-sub bit', 'coals', 'ciak', 'petcoke', 'coal.oil', 'coal/gas',
        'bit coal', 'coal-unit #3', 'coal-subbitum', 'coal tons', 'coal mcf',
        'coal unit #3', 'pet. coke', 'coal-u3', 'coal&coke', 'tons'
    ],
    "oil": [
        'oil', '#6 oil', '#2 oil', 'fuel oil', 'jet', 'no. 2 oil', 'no.2 oil',
        'no.6& used', 'used oil', 'oil-2', 'oil (#2)', 'diesel oil',
        'residual oil', '# 2 oil', 'resid. oil', 'tall oil', 'oil/gas',
        'no.6 oil', 'oil-fuel', 'oil-diesel', 'oil / gas', 'oil bbls', 'oil bls',
        'no. 6 oil', '#1 kerosene', 'diesel', 'no. 2 oils', 'blend oil',
        '#2oil diesel', '#2 oil-diesel', '# 2  oil', 'light oil', 'heavy oil',
        'gas.oil', '#2', '2', '6', 'bbl', 'no 2 oil', 'no 6 oil', '#1 oil', '#6',
        'oil-kero', 'oil bbl', 'biofuel', 'no 2', 'kero', '#1 fuel oil',
        'no. 2  oil', 'blended oil', 'no 2. oil', '# 6 oil', 'nno. 2 oil',
        '#2 fuel', 'oill', 'oils', 'gas/oil', 'no.2 oil gas', '#2 fuel oil',
        'oli', 'oil (#6)', 'oil/diesel', '2 oil', '#6 hvy oil', 'jet fuel',
        'diesel/compos', 'oil-8', 'oil {6}', 'oil-unit #1', 'bbl.', 'oil.',  # noqa: FS003
        'oil #6', 'oil (6)', 'oil(#2)', 'oil-unit1&2', 'oil-6', '#2 fue oil',
        'dielel oil', 'dielsel oil', '#6 & used', 'barrels', 'oil un 1 & 2',
        'jet oil', 'oil-u1&2', 'oiul', 'pil', 'oil - 2', '#6 & used', 'oial'
    ],
    "gas": [
        'gas', 'gass', 'methane', 'natural gas', 'blast gas', 'gas mcf',
        'propane', 'prop', 'natural  gas', 'nat.gas', 'nat gas',
        'nat. gas', 'natl gas', 'ga', 'gas`', 'syngas', 'ng', 'mcf',
        'blast gaa', 'nat  gas', 'gac', 'syngass', 'prop.', 'natural', 'coal.gas',
        'n. gas', 'lp gas', 'natuaral gas', 'coke gas', 'gas #2016', 'propane**',
        '* propane', 'propane **', 'gas expander', 'gas ct', '# 6 gas', '#6 gas',
        'coke oven gas'
    ],
    "solar": [],
    "wind": [],
    "hydro": [],
    "nuclear": [
        'nuclear', 'grams of uran', 'grams of', 'grams of  ura',
        'grams', 'nucleur', 'nulear', 'nucl', 'nucleart', 'nucelar',
        'gr.uranium', 'grams of urm', 'nuclear (9)', 'nulcear', 'nuc',
        'gr. uranium', 'nuclear mw da', 'grams of ura'
    ],
    "waste": [
        'tires', 'tire', 'refuse', 'switchgrass', 'wood waste', 'woodchips',
        'biomass', 'wood', 'wood chips', 'rdf', 'tires/refuse', 'tire refuse',
        'waste oil', 'waste', 'woodships', 'tire chips'
    ],
    "unknown": [
        'steam', 'purch steam', 'all', 'tdf', 'n/a', 'purch. steam', 'other',
        'composite', 'composit', 'mbtus', 'total', 'avg', 'avg.', 'blo',
        'all fuel', 'comb.', 'alt. fuels', 'na', 'comb', '/#=2\x80â\x91?',
        'kã\xadgv¸\x9d?', "mbtu's", 'gas, oil', 'rrm', '3\x9c', 'average',
        'furfural', '0', 'watson bng', 'toal', 'bng', '# 6 & used', 'combined',
        'blo bls', 'compsite', '*', 'compos.', 'gas / oil', 'mw days', 'g', 'c',
        'lime', 'all fuels', 'at right', '20', '1', 'comp oil/gas', 'all fuels to',
        'the right are', 'c omposite', 'all fuels are', 'total pr crk',
        'all fuels =', 'total pc', 'comp', 'alternative', 'alt. fuel', 'bio fuel',
        'total prairie', ''
    ],
}
"""dict: A mapping a canonical fuel name to a list of strings which are used
to represent that fuel in the FERC Form 1 Reporting. Case is ignored, as all fuel
strings are converted to a lower case in the data set.
"""

FUEL_UNIT_STRINGS = {
    "ton": [
        'toms', 'taons', 'tones', 'col-tons', 'toncoaleq', 'coal', 'tons coal eq',
        'coal-tons', 'ton', 'tons', 'tons coal', 'coal-ton', 'tires-tons',
        'coal tons -2 ', 'oil-tons', 'coal tons 200', 'ton-2000', 'coal tons',
        'coal tons -2', 'coal-tone', 'tire-ton', 'tire-tons', 'ton coal eqv', 'tos',
        'coal tons - 2', 'c. t.', 'c.t.', 'toncoalequiv',
    ],
    "mcf": [
        'mcf', "mcf's", 'mcfs', 'mcf.', 'gas mcf', '"gas" mcf', 'gas-mcf',
        'mfc', 'mct', ' mcf', 'msfs', 'mlf', 'mscf', 'mci', 'mcl', 'mcg',
        'm.cu.ft.', 'kcf', '(mcf)', 'mcf *(4)', 'mcf00', 'm.cu.ft..',
    ],
    "bbl": [
        'barrel', 'bbls', 'bbl', 'barrels', 'bbrl', 'bbl.', 'bbls.', 'oil 42 gal',
        'oil-barrels', 'barrrels', 'bbl-42 gal', 'oil-barrel', 'bb.', 'barrells',
        'bar', 'bbld', 'oil- barrel', 'barrels    .', 'bbl .', 'barels', 'barrell',
        'berrels', 'bb', 'bbl.s', 'oil-bbl', 'bls', 'bbl:', 'barrles', 'blb',
        'propane-bbl', 'barriel', 'berriel', 'barrile', '(bbl.)', 'barrel *(4)',
        '(4) barrel', 'bbf', 'blb.', '(bbl)', 'bb1', 'bbsl', 'barrrel', 'barrels 100%',
        'bsrrels', "bbl's", '*barrels', 'oil - barrels', 'oil 42 gal ba', 'bll',
        'boiler barrel', 'gas barrel', '"boiler" barr', '"gas" barrel',
        '"boiler"barre', '"boiler barre', 'barrels .', 'bariel', 'brrels', 'oil barrel',
    ],
    "gal": ['gallons', 'gal.', 'gals', 'gals.', 'gallon', 'gal', 'galllons'],
    "kgal": ['oil(1000 gal)', 'oil(1000)', 'oil (1000)', 'oil(1000', 'oil(1000ga)'],
    "gramsU": [
        'gram', 'grams', 'gm u', 'grams u235', 'grams u-235', 'grams of uran',
        'grams: u-235', 'grams:u-235', 'grams:u235', 'grams u308', 'grams: u235',
        'grams of', 'grams - n/a', 'gms uran', 's e uo2 grams', 'gms uranium',
        'grams of urm', 'gms. of uran', 'grams (100%)', 'grams v-235', 'se uo2 grams',
    ],
    "kgU": [
        'kg of uranium', 'kg uranium', 'kilg. u-235', 'kg u-235', 'kilograms-u23',
        'kg', 'kilograms u-2', 'kilograms', 'kg of', 'kg-u-235', 'kilgrams',
        'kilogr. u235', 'uranium kg', 'kg uranium25', 'kilogr. u-235',
        'kg uranium 25', 'kilgr. u-235', 'kguranium 25', 'kg-u235', 'kgm',
    ],
    "klbs": ['k lbs.', 'k lbs'],
    "mmbtu": [
        'mmbtu', 'mmbtus', 'mbtus', '(mmbtu)', "mmbtu's", 'nuclear-mmbtu',
        'nuclear-mmbt', 'mmbtul',
    ],
    "btu": ['btus', 'btu', ],
    "mwdth": [
        'mwd therman', 'mw days-therm', 'mwd thrml', 'mwd thermal',
        'mwd/mtu', 'mw days', 'mwdth', 'mwd', 'mw day', 'dth', 'mwdaysthermal',
        'mw day therml', 'mw days thrml', 'nuclear mwd', 'mmwd', 'mw day/therml'
        'mw days/therm', 'mw days (th', 'ermal)',
    ],
    "mwhth": [
        'mwh them', 'mwh threm', 'nwh therm', 'mwhth',
        'mwh therm', 'mwh', 'mwh therms.', 'mwh term.uts',
        'mwh thermal', 'mwh thermals', 'mw hr therm',
        'mwh therma', 'mwh therm.uts',
    ],
    "unknown": [
        '', '1265', 'mwh units', 'composite', 'therms', 'n/a', 'mbtu/kg', 'uranium 235',
        'oil', 'ccf', '2261', 'uo2', '(7)', 'oil #2', 'oil #6', '\x99å\x83\x90?"',
        'dekatherm', '0', 'mw day/therml', 'nuclear', 'gas', '62,679', 'mw days/therm',
        'na', 'uranium', 'oil/gas', 'thermal', '(thermal)', 'se uo2', '181679', '83',
        '3070', '248', '273976', '747', '-', 'are total', 'pr. creek', 'decatherms',
        'uramium', '.', 'total pr crk', '>>>>>>>>', 'all', 'total', 'alternative-t',
        'oil-mcf', '3303671', '929', '7182175', '319', '1490442', '10881', '1363663',
        '7171', '1726497', '4783', '7800', '12559', '2398', 'creek fuels',
        'propane-barre', '509', 'barrels/mcf', 'propane-bar', '4853325', '4069628',
        '1431536', '708903', 'mcf/oil (1000',
    ],
}
"""
dict: A dictionary linking fuel units (keys) to lists of various strings
    representing those fuel units (values)
"""

'\ndict: A dictionary linking fuel units (keys) to lists of various strings\n    representing those fuel units (values)\n'